In [117]:
import requests, datetime, time
from bs4 import BeautifulSoup

def httpReq(board, indexNum):
    index_url = 'https://www.ptt.cc/bbs/' + board + '/index' + str(indexNum) + '.html'
    index_res = requests.get(index_url)
    return BeautifulSoup(index_res.text, 'html5lib')

def link_list(board, indexNum):
    link_list = []
    for link in httpReq(board, indexNum).select('.r-ent'):
        link_list.append('https://www.ptt.cc' + link.select('a')[0]['href'])
    return link_list

def grap_push_info(soup):
    pushInfo_list = []
    for i in soup.select('.push'):
        if len(i.select('.push-ipdatetime')[0].string.strip()) > 11:
            continue
        pushInfo_list.append(
                              # date (e.g. '02/18')
                             [i.select('.push-ipdatetime')[0].string.strip(), 
                              # userid (e.g. 'comdan66')
                              i.select('.push-userid')[0].string.strip(), 
                              # push content
                              i.select('.push-content')[0].string[1:].strip()]
        )
    return pushInfo_list

def grap_1F(soup):
    return (
        # date (e.g. 'Mon Aug 29 14:32:25 2005')
        soup.select('.article-meta-value')[3].text.strip() +'``!@'+
        # userid (e.g. 'skyboy')
        soup.select('.article-meta-value')[0].text.split('(')[0].strip() +'``!@'+                   
        # content
        soup.select('#main-content')[0].contents[4].strip()
    ).split('``!@')

def get_year(soup, push_m, push_d):
    post_y = int(soup.select('.article-meta-value')[3].text.strip()[-4:])
    if push_m == 12:
        return str(post_y)
    elif (abs(datetime.date(post_y, 12, 31) - datetime.date(post_y+1, push_m, push_d)) < datetime.timedelta(days=180)):
        return str(post_y+1)
    else:
        return str(post_y)

def grap_content():
    for link in link_list('Yunlin',7):
        con_res = requests.get(link)
        con_soup = BeautifulSoup(con_res.text, 'html5lib')
#        try:
        # crawl main content
        print(
              # date (e.g. 'Mon Aug 29 14:32:25 2005')
              grap_1F(con_soup)[0],
              # userid (e.g. 'skyboy')
              grap_1F(con_soup)[1],
              # content
              grap_1F(con_soup)[2],
              # url
              con_res.url,
              # is main content?
              'True'
        )

        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

        # crawl push content
        for i in grap_push_info(con_soup):
            print(get_year(con_soup, int(i[0][0:2]), int(i[0][3:5])) + '/' + i[0], i[1], i[2], con_res.url, 'False')
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
#        except:
#            continue
            
grap_content()



Thu Dec 15 00:53:40 2005 fdps0808 哈!
我們斗中人來PTT的人數也越來越多了,
但不是每個班級都有班版,
所以開了一個校友版暨組務版(GreenCity),
統一處理斗六高中群組內的事務,
以及各位斗中校友,不管是學長姊,學弟妹,
或仍在校的學弟妹一個駐足的地方!
歡迎各位學長姊,學弟妹有空到那坐坐~
^^

                    小組長   fdps0808

--
綠城品質  堅若磐石
--

-- https://www.ptt.cc/bbs/Yunlin/M.1134579183.A.9CF.html True
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
2005/12/16 20:19 weigi 斗六高中簡稱斗中嗎? 我都稱斗高耶 https://www.ptt.cc/bbs/Yunlin/M.1134579183.A.9CF.html False
2005/12/16 21:15 fdps0808 都有人說,我通常都兩個互用,隨機~ https://www.ptt.cc/bbs/Yunlin/M.1134579183.A.9CF.html False
2005/12/16 21:15 fdps0808 教官啊,主任啊還說雲林一中咧.....XD https://www.ptt.cc/bbs/Yunlin/M.1134579183.A.9CF.html False
2005/12/17 09:29 jayclub 以前在斗六國中時自稱斗中，後來上高中還是自稱斗中...XD https://www.ptt.cc/bbs/Yunlin/M.1134579183.A.9CF.html False
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Fri Dec 16 18:26:19 2005 cloudko 我看過精華區嚕
都沒人問過此問題
因為明天要去雲科考試
所以想請問有沒有雲科的同學知道從火車

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [117]:
import datetime
day1 = datetime.date(2015,1,1)
day2 = datetime.date(2015,12,31)
post_y = 2005

a = abs(datetime.date(post_y, 12, 31) - datetime.date(2006,1,1))
b = abs(datetime.date(post_y + 1, 12, 31) - datetime.date(2005, 1, 1))
print(a< datetime.timedelta(days=180))


def get_year(soup, push_m, push_d):
    post_y = int(soup.select('.article-meta-value')[3].text.strip()[-4:])
    if push_m <= 6 or push_m == 12:
        return str(post_y)
    elif (abs(datetime.date(post_y, 12, 31) - datetime.date(post_y, push_m, push_d)) < datetime.timedelta(days=180)):
        return str(post_y+1)
    else:
        return str(post_y)

True
